In [1]:
import pandas as pd
from footix.models.bayesian import Bayesian
import footix.implied_odds as odds
import numpy as np
from footix.data_io.data_scrapper import ScrapFootballData
from footix.strategy.strategies import realKelly, simple_select_bets

In [2]:
dataset = ScrapFootballData(competition="ligue1", season="2425", path ="./data", force_reload=True).get_data()

In [3]:
dataset.head(-5)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA
0,F1,16/08/2024,19:45,Le Havre,Paris SG,1,4,A,0,1,...,1.93,2.00,1.95,1.97,1.95,2.02,1.89,1.95,1.95,2.02
1,F1,17/08/2024,16:00,Brest,Marseille,1,5,A,1,3,...,1.89,2.04,1.91,2.02,1.91,2.08,1.85,2.01,1.90,2.09
2,F1,17/08/2024,18:00,Reims,Lille,0,2,A,0,1,...,2.10,1.70,2.14,1.81,2.14,1.85,2.07,1.80,2.13,1.86
3,F1,17/08/2024,20:00,Monaco,St Etienne,1,0,H,1,0,...,2.00,1.93,2.00,1.93,2.00,2.01,1.95,1.91,1.95,2.00
4,F1,18/08/2024,14:00,Auxerre,Nice,2,1,H,1,1,...,1.82,2.11,1.82,2.13,1.83,2.17,1.77,2.10,1.84,2.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,F1,13/04/2025,19:45,Auxerre,Lyon,1,3,A,0,0,...,1.90,2.03,1.86,2.07,1.90,2.07,1.83,2.03,1.89,2.10
259,F1,18/04/2025,19:45,Rennes,Nantes,2,1,H,1,0,...,1.89,2.01,1.88,2.05,1.92,2.15,1.84,2.02,1.89,2.11
260,F1,19/04/2025,16:00,Paris SG,Le Havre,2,1,H,1,0,...,2.02,1.91,2.01,1.90,2.05,1.91,1.98,1.87,2.03,1.96
261,F1,19/04/2025,18:00,Monaco,Strasbourg,0,0,D,0,0,...,2.03,1.90,2.03,1.90,2.07,1.90,2.02,1.84,2.10,1.88


In [4]:
dataset["HomeTeam"].unique()

array(['Le Havre', 'Brest', 'Reims', 'Monaco', 'Auxerre', 'Angers',
       'Montpellier', 'Toulouse', 'Rennes', 'Paris SG', 'Lyon', 'Lille',
       'St Etienne', 'Lens', 'Nantes', 'Nice', 'Strasbourg', 'Marseille'],
      dtype=object)

In [5]:
model = Bayesian(n_teams=18, n_goals=15)

In [ ]:
model.fit(X_train=dataset)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (6 chains in 6 jobs)
NUTS: [home, intercept, tau_att, raw_atts, tau_def, raw_defs]


Output()

In [ ]:
fixture = pd.read_csv("ligue1_18-04-2025.csv")
# Assuming fixture and odds are already defined DataFrames
proba_match = []
for idx, match in fixture.iterrows():
    print("#" * 50)
    print(f"{match['home_team']:<30} {match['away_team']:<30}")
    # Display probabilities from Poisson prediction
    probas = model.predict(home_team=match["home_team"], away_team=match["away_team"]).return_probas()
    proba_match.append(probas)
    home_prob = probas[0]
    draw_prob = probas[1]
    away_prob = probas[2]
    print(f"  Probabilities: Home: {home_prob:.2f}, Draw: {draw_prob:.2f}, Away: {away_prob:.2f}")
    # Display odds from the function
    odds_list = [match["H"], match["D"], match["A"]]
    odds_result = odds.shin(odds=odds_list)[0]
    print(f"  Odds: Home: {odds_result[0]:.2f}, Draw: {odds_result[1]:.2f}, Away: {odds_result[2]:.2f}")
    print("#" * 50)
proba_match = np.asarray(proba_match)

##################################################
Rennes                         Nantes                        
  Probabilities: Home: 0.45, Draw: 0.26, Away: 0.29
  Odds: Home: 0.53, Draw: 0.25, Away: 0.21
##################################################
##################################################
Paris SG                       Le Havre                      
  Probabilities: Home: 0.82, Draw: 0.11, Away: 0.07
  Odds: Home: 0.85, Draw: 0.10, Away: 0.05
##################################################
##################################################
Monaco                         Strasbourg                    
  Probabilities: Home: 0.55, Draw: 0.22, Away: 0.23
  Odds: Home: 0.63, Draw: 0.20, Away: 0.16
##################################################
##################################################
Marseille                      Montpellier                   
  Probabilities: Home: 0.82, Draw: 0.12, Away: 0.07
  Odds: Home: 0.81, Draw: 0.13, Away: 0.07
###############

In [ ]:
list_bet = simple_select_bets(odds_bookie=fixture, probas=proba_match, one_bet_game=True)

In [ ]:
realKelly(list_bet[:9], bankroll=1000, max_multiple=1, early_stopping=True)

  0%|          | 0/1000 [00:00<?, ?it/s]


2025-04-21 12:22:30- Optimization finished. Runtime --- 1.437 seconds ---

Objective: -6.944
Certainty Equivalent: 1036.912

[Rennes - Nantes | A] odds=4.40, edge=0.289, p=0.293, stake=55.34
[Paris SG - Le Havre | A] odds=15.00, edge=0.107, p=0.074, stake=3.47
[Monaco - Strasbourg | A] odds=5.40, edge=0.232, p=0.228, stake=33.98
[Lille - Auxerre | A] odds=6.40, edge=0.396, p=0.218, stake=50.04
[Nice - Angers | A] odds=7.50, edge=0.240, p=0.165, stake=21.70
[Brest - Lens | A] odds=3.00, edge=0.054, p=0.351, stake=9.89
[Paris SG - Nice | A] odds=6.20, edge=0.078, p=0.174, stake=5.85
[Rennes - Nantes+Paris SG - Le Havre | A+A] odds=66.00, edge=0.427, p=0.022, stake=1.06
[Rennes - Nantes+Monaco - Strasbourg | A+A] odds=23.76, edge=0.589, p=0.067, stake=5.64
[Rennes - Nantes+Lille - Auxerre | A+D] odds=18.92, edge=0.295, p=0.068, stake=1.08
[Rennes - Nantes+Lille - Auxerre | A+A] odds=28.16, edge=0.800, p=0.064, stake=7.33
[Rennes - Nantes+Nice - Angers | A+D] odds=21.56, edge=0.293, p=0.0

[Bet(match_id='Rennes - Nantes', market='A', odds=4.4, edge_mean=0.2893570577912409, prob_mean=0.2930356949525547, edge_std=None, prob_edge_pos=None, stake=55.34429931640625),
 Bet(match_id='Paris SG - Le Havre', market='A', odds=15.0, edge_mean=0.10693095775209371, prob_mean=0.07379539718347292, edge_std=None, prob_edge_pos=None, stake=3.467168092727661),
 Bet(match_id='Monaco - Strasbourg', market='A', odds=5.4, edge_mean=0.23209993257043093, prob_mean=0.2281666541797094, edge_std=None, prob_edge_pos=None, stake=33.98339080810547),
 Bet(match_id='Lille - Auxerre', market='A', odds=6.4, edge_mean=0.3957303940634084, prob_mean=0.21808287407240753, edge_std=None, prob_edge_pos=None, stake=50.03962326049805),
 Bet(match_id='Nice - Angers', market='A', odds=7.5, edge_mean=0.24036871880094202, prob_mean=0.1653824958401256, edge_std=None, prob_edge_pos=None, stake=21.704666137695312),
 Bet(match_id='Brest - Lens', market='A', odds=3.0, edge_mean=0.053692373943191485, prob_mean=0.35123079131

In [ ]:
#pm.summary(model.trace)

In [20]:
# Assuming fixture and odds are already defined DataFrames
fixture = pd.read_csv("odds_info.csv")
selections = []

for idx, match in fixture.iterrows():
    home_team = match["home_team"]
    away_team = match["away_team"]
    name = f"{home_team} vs {away_team} - "

    # Display probabilities from Poisson prediction
    probas = model.predict(home_team=home_team, away_team=away_team).return_probas()
    home_prob = probas[0]
    draw_prob = probas[1]
    away_prob = probas[2]

    # Display odds from the function
    odds_list = [match["H"], match["D"], match["A"]]
    odds_result = odds.shin(odds=odds_list)[0]
    home_implied = odds_result[0]
    draw_implied = odds_result[1]
    away_implied = odds_result[2]

    home_odd = odds_list[0]
    draw_odd = odds_list[1]
    away_odd = odds_list[2]

    # Calculate the value for each outcome
    home_value = home_prob - home_implied
    draw_value = draw_prob - draw_implied
    away_value = away_prob - away_implied

    # Determine the best selection
    if home_value >= draw_value and home_value >= away_value:
        best_prob = home_prob
        best_odd = home_odd
        sel = "home"
    elif draw_value >= home_value and draw_value >= away_value:
        best_prob = draw_prob
        best_odd = draw_odd
        sel = "draw"
    else:
        best_prob = away_prob
        best_odd = away_odd
        sel = "away"
    # Append the selection to the list
    selections.append({
        "name": name+sel,
        "probability": best_prob,
        "odds_bookie": best_odd
    })

In [21]:
print(selections)

[{'name': 'Rennes vs Nantes - away', 'probability': 0.30277683589466087, 'odds_bookie': 4.45}, {'name': 'Paris SG vs Le Havre - away', 'probability': 0.06464313101722349, 'odds_bookie': 16.0}, {'name': 'Monaco vs Strasbourg - away', 'probability': 0.21752235242031678, 'odds_bookie': 5.1}, {'name': 'Marseille vs Montpellier - draw', 'probability': 0.1383516745906119, 'odds_bookie': 7.0}, {'name': 'Lille vs Auxerre - away', 'probability': 0.23505071892226573, 'odds_bookie': 6.2}, {'name': 'Nice vs Angers - away', 'probability': 0.16377698966446522, 'odds_bookie': 7.6}, {'name': 'Brest vs Lens - away', 'probability': 0.31748628634625736, 'odds_bookie': 3.0}, {'name': 'Reims vs Toulouse - away', 'probability': 0.4717547202016954, 'odds_bookie': 2.1}, {'name': 'St Etienne vs Lyon - away', 'probability': 0.6159054186063452, 'odds_bookie': 1.55}, {'name': 'Nantes vs Paris SG - away', 'probability': 0.7438752173474351, 'odds_bookie': 1.33}, {'name': 'Paris SG vs Nice - away', 'probability': 0.

In [21]:
realKelly(selections[:9], bankroll=158, max_multiple=3, device="cpu", early_stopping=False, num_iterations=5_000)

NameError: name 'selections' is not defined